$Expected FTB Booking and #Expected FTB Accounts

For each representative (opportunity owner) in Pre-FTB report, collect all the accounts under their name, together with Expected FTB Date (simply 'date') and Annual hotel spend (simply 'spend').

Starting from the date, split the spend over the following twelve months. Start with the month the date belongs to. (There are some strange cases, e.g. Adam Ward, Frontier Building Corp, FTB date = 30 Nov, i.e. the end of the month already, but Nov takes one entire 12th part.)

Some accounts don't have a date. I'm ignoring those.

For each month, sum these fractions over all the accounts. Also count the number of accounts. These numbers are the $Expected FTB Booking and #Expected FTB Accounts respectively, and they're shown in the Pipeline Summary.

Treat 'post' data in the same way.

There's a little difference in the number of post accounts. That's because i don't count accounts with $0.

In [ ]:
# Exported

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as mticker

from datetime import datetime, timedelta
from utils_dates import find_date_range, extract_date

from utils_analysis import customer_detail, partial_pipeline_summary

from utils_visualizations import vialualize_partial_pipeline, vialualize_total_pipeline

In [ ]:
# Exported

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

# Filter or ignore the warning
import warnings
warnings.filterwarnings("ignore", category=pd.core.common.SettingWithCopyWarning)

# Import Data

In [ ]:
df_pre = pd.read_csv('original_files/Pre-FTB_Report.csv')
df_post = pd.read_csv('original_files/Post-FTB_Report.csv')

df_pre['Annual hotel spend'] = df_pre['Annual hotel spend'].fillna(0).astype(int)
df_post['Annual hotel spend'] = df_post['Annual hotel spend'].fillna(0).astype(int)

In [ ]:
# difference in the input files from pre to post
# the FTB includes both date and time
df_pre[:3]

In [ ]:
df_post[:3]

In [ ]:
#  we just need the date
df_post['Expected FTB Date'] = [extract_date(date_time) for date_time in df_post['FTB or Revival Date'] ]
df_post = df_post.drop('FTB or Revival Date', axis=1)
df_post[:3]

# Owners

In [ ]:
pre_owners = np.sort(np.array(df_pre['Opportunity Owner'].unique()))
post_owners = np.sort(np.array(df_post['Opportunity Owner'].unique()))

missing_owners = [owner for owner in pre_owners if owner not in post_owners]

In [ ]:
missing_owners

In [ ]:
print(len(pre_owners))
print(len(post_owners))

In [ ]:
# I will only keep owners that are in both pre and post reports

# Convert arrays to sets
set1 = set(pre_owners)
set2 = set(post_owners)

# Find the intersection of the sets
common_elements = set1.intersection(set2)

# Convert the common elements set back to a list
names = list(common_elements)

# sort it alphabetically by family name
owners_list = sorted(names, key=lambda x: x.split()[1])

#owners_list

# Partial Visualizations

In [ ]:
ownerID = 30
owner = owners_list[ownerID]

detail_pre = customer_detail(df_pre,owner)
detail_post = customer_detail(df_post,owner)

print(owner)

In [ ]:
detail_pre[:2]

In [ ]:
detail_post[:2]

In [ ]:
partial_pipeline_summary( detail_pre , detail_type = 'pre' )

In [ ]:
partial_pipeline_summary( detail_post , detail_type = 'post' )

In [ ]:
vialualize_partial_pipeline (detail_pre,detail_type='pre')

In [ ]:
vialualize_partial_pipeline (detail_post,detail_type='post')

# Pipeline

In [ ]:
summ_pre = partial_pipeline_summary( detail_pre , detail_type = 'pre' )
summ_post = partial_pipeline_summary( detail_post , detail_type = 'post' )

In [ ]:
merged_df = pd.concat([summ_post,summ_pre], sort=False).fillna(0)
# Calculate the sum of pre and post
row_sum = merged_df.iloc[0] + merged_df.iloc[2]
# Set a name for the new row
row_sum.name = '$ Total Pipeline'
# Append the sum as a new row to the dataframe
merged_df = merged_df.append(row_sum)

In [ ]:
dfcommas = merged_df.applymap('{:,.0f}'.format)
dfcommas

In [ ]:
df = dfcommas


# Define a dictionary of custom colors for specific rows
custom_colors = {#'$ Confirmed Pipeline Bookings': 'white', 
                 #'# Confirmed Pipeline Accounts': 'white',
                #'$ Expected FTB Booking': 'lightgray', 
                 #'# Expected FTB Accounts': 'white',
                 '$ Total Pipeline':'lightyellow'
                }

# Function to apply custom colors to rows
def apply_custom_colors(row):
    row_index = row.name
    if row_index in custom_colors:
        return ['background-color: {}'.format(custom_colors[row_index])] * len(row)
    else:
        return [''] * len(row)
    

# Apply the custom colors to the dataframe
styled_df = df.style.apply(apply_custom_colors, axis=1)

# Display the styled dataframe
styled_df

In [ ]:
df_total = merged_df.loc['$ Total Pipeline']
df_expected = merged_df.loc['$ Expected FTB Booking']

In [ ]:
vialualize_total_pipeline (df_total,df_expected)

To do:
1. Some numbers don't coincide. For example: Andrew Shock (ID = 30)
2. Figure out the lines "Pipeline Coverage Multiple" and "AE Quota", then add them to the final dataframe